## Imports

In [57]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from datetime import datetime

## Load `train.csv`

In [58]:
df_train: pd.DataFrame = pd.read_csv("data/train.csv")

df_test: pd.DataFrame = pd.read_csv("data/test_public.csv")

In [59]:
t = set(df_train["ORIGIN_CALL"].unique())
s = set(df_test["ORIGIN_CALL"].unique())

In [60]:
len(t)

57106

In [61]:
df_train.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."


## Delete rows where `MISSING_DATA` is true and delete column `MISSING_DATA`

In [62]:
df_train.drop(df_train[df_train["MISSING_DATA"] == True].index, inplace=True)
df_train.drop(labels="MISSING_DATA", axis=1, inplace=True)

df_test.drop(df_test[df_test["MISSING_DATA"] == True].index, inplace=True)
df_test.drop(labels="MISSING_DATA", axis=1, inplace=True)

## Delete column `DAY_TYPE`

In [63]:
df_train.drop(labels="DAY_TYPE", axis=1, inplace=True)

df_test.drop(labels="DAY_TYPE", axis=1, inplace=True)

## Delete column `TRIP_ID`

In [64]:
df_train.drop(labels="TRIP_ID", axis=1, inplace=True)

df_test.drop(labels="TRIP_ID", axis=1, inplace=True)

## Set `ORIGIN_STAND` and `ORIGIN_CALL` to 0 for entries with null `ORIGIN_STAND`

In [65]:
df_train.at[pd.isna(df_train["ORIGIN_CALL"]) == True, "ORIGIN_CALL"] = 0
df_train.at[pd.isna(df_train["ORIGIN_STAND"]) == True, "ORIGIN_STAND"] = 0

df_test.at[pd.isna(df_test["ORIGIN_CALL"]) == True, "ORIGIN_CALL"] = 0
df_test.at[pd.isna(df_test["ORIGIN_STAND"]) == True, "ORIGIN_STAND"] = 0

# df_train.drop(labels="ORIGIN_CALL", axis=1, inplace=True)
# df_test.drop(labels="ORIGIN_CALL", axis=1, inplace=True)

### Convert ORIGIN_CALL to indices for embeddings

In [72]:
# NOTE: EMBEDDING INFO STUFF
# TODO: Convert ORIGIN_CALL to embeddings, impute missing values
# Look for caller's with 1 total taxi ride, set them all to some default index (Training set)
# Look for callers in test set not in training set -> set them to the default index
# Set nn.embeddings pad_idx to that default idx
val_cnt = df_train["ORIGIN_CALL"].value_counts().to_dict()
df_train["ORIGIN_CALL"] = df_train["ORIGIN_CALL"].map(lambda x : x  if val_cnt[x] > 1 else 0 )

In [77]:
df_train["ORIGIN_CALL"].value_counts()

0.0        1373970
2002.0       57571
63882.0       6406
2001.0        2499
13168.0       1314
            ...   
63189.0          2
50313.0          2
53254.0          2
3675.0           2
62998.0          2
Name: ORIGIN_CALL, Length: 29027, dtype: int64

In [76]:
multi_occurences = set(df_train["ORIGIN_CALL"].unique())
new_col = df_test["ORIGIN_CALL"]
for i, x in enumerate(new_col):
    if x not in multi_occurences:
        new_col[i] = 0
df_test["ORIGIN_CALL"] = new_col

C:\Users\benjx\AppData\Local\Temp/ipykernel_10964/1392256516.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_col[i] = 0


In [78]:
LE_ORIGIN_CALL = LabelEncoder()
LE_ORIGIN_CALL.fit(df_train["ORIGIN_CALL"])
df_train["ORIGIN_CALL"] = LE_ORIGIN_CALL.transform(df_train["ORIGIN_CALL"])

LE_ORIGIN_STAND = LabelEncoder()
LE_ORIGIN_STAND.fit(df_train["ORIGIN_STAND"])
df_train["ORIGIN_STAND"] = LE_ORIGIN_STAND.transform(df_train["ORIGIN_STAND"])


df_test["ORIGIN_CALL"] = LE_ORIGIN_CALL.transform(df_test["ORIGIN_CALL"])
df_test["ORIGIN_STAND"] = LE_ORIGIN_STAND.transform(df_test["ORIGIN_STAND"])

## One-hot encoding for `CALL_TYPE`

In [79]:
# one-hot encoding
df_train = df_train.join(pd.get_dummies(df_train["CALL_TYPE"]))
df_train.drop(labels="CALL_TYPE", axis=1, inplace=True)

df_test = df_test.join(pd.get_dummies(df_test["CALL_TYPE"]))
df_test.drop(labels="CALL_TYPE", axis=1, inplace=True)

## Assign indices to TAXI_ID for embeddings

In [80]:
# TODO: set taxi id to indices
LE_TAXI_ID = LabelEncoder()
LE_TAXI_ID.fit(df_train["TAXI_ID"])
df_train["TAXI_ID"] = LE_TAXI_ID.transform(df_train["TAXI_ID"])

df_test["TAXI_ID"] = LE_TAXI_ID.transform(df_test["TAXI_ID"])

In [81]:
df_train.head()

,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,POLYLINE,A,B,C
0,0,0,367,1372636858,"[[-8.618643,41.141412],[-8.618499,41.141376],[...",0,0,1
1,0,7,371,1372637303,"[[-8.639847,41.159826],[-8.640351,41.159871],[...",0,1,0
2,0,0,204,1372636951,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...",0,0,1
3,0,0,330,1372636854,"[[-8.574678,41.151951],[-8.574705,41.151942],[...",0,0,1
4,0,0,217,1372637091,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...",0,0,1


## Convert `TIMESTAMP` column to quarter hours, week of year, etc.

In [82]:
df_train["YEAR"] = df_train["TIMESTAMP"].apply(lambda x: datetime.fromtimestamp(x).year)
df_train["WK_OF_YR"] = df_train["TIMESTAMP"].apply(lambda x: datetime.fromtimestamp(x).isocalendar().week)
df_train["WK_DAY"] = df_train["TIMESTAMP"].apply(lambda x: datetime.fromtimestamp(x).weekday()) # WHY ARE YOU LIKE THIS??? ALL THE OTHERS ARE FIELDS
df_train["QTR_HR"] = df_train["TIMESTAMP"].apply(lambda x: datetime.fromtimestamp(x).hour + 0.25 * (datetime.fromtimestamp(x).minute // 15))

df_train.drop(labels="TIMESTAMP", axis=1, inplace=True)


df_test["YEAR"] = df_test["TIMESTAMP"].apply(lambda x: datetime.fromtimestamp(x).year)
df_test["WK_OF_YR"] = df_test["TIMESTAMP"].apply(lambda x: datetime.fromtimestamp(x).isocalendar().week)
df_test["WK_DAY"] = df_test["TIMESTAMP"].apply(lambda x: datetime.fromtimestamp(x).weekday()) # WHY ARE YOU LIKE THIS??? ALL THE OTHERS ARE FIELDS
df_test["QTR_HR"] = df_test["TIMESTAMP"].apply(lambda x: datetime.fromtimestamp(x).hour + 0.25 * (datetime.fromtimestamp(x).minute // 15))

df_test.drop(labels="TIMESTAMP", axis=1, inplace=True)


In [83]:
LE_WK_OF_YR = LabelEncoder()
LE_WK_OF_YR.fit(df_train["WK_OF_YR"])
df_train["WK_OF_YR"] = LE_WK_OF_YR.transform(df_train["WK_OF_YR"])

LE_WK_DAY = LabelEncoder()
LE_WK_DAY.fit(df_train["WK_DAY"])
df_train["WK_DAY"] = LE_WK_DAY.transform(df_train["WK_DAY"])

LE_QTR_HR = LabelEncoder()
LE_QTR_HR.fit(df_train["QTR_HR"])
df_train["QTR_HR"] = LE_QTR_HR.transform(df_train["QTR_HR"])

In [ ]:
df_train.head()

## Convert `POLYLINE` column to `TARGET` column.

In [84]:
df_train["TARGET"] = df_train["POLYLINE"].apply(lambda x : 15 * max(x.count("[") - 1, 0))
df_train.drop(labels="POLYLINE", axis=1, inplace=True)

# wtf am i doing
# df_test["TARGET"] = df_test["POLYLINE"].apply(lambda x : 15 * max(x.count("[") - 1, 0))
# df_test.drop(labels="POLYLINE", axis=1, inplace=True)

## Get information for embeddings later

In [85]:
print(f"CALL_TYPE: 3")
print(f"ORIGIN_CALL: {len(df_train['ORIGIN_CALL'].unique())}")
print(f"ORIGIN_STAND: {len(df_train['ORIGIN_STAND'].unique())}")
print(f"TAXI_ID: {len(df_train['TAXI_ID'].unique())}")
print(f"Unique week #: {len(df_train['WK_OF_YR'].unique())}")
print(f"Unique qtr hrs #: {len(df_train['QTR_HR'].unique())}")

CALL_TYPE: 3
ORIGIN_CALL: 29027
ORIGIN_STAND: 64
TAXI_ID: 448
Unique week #: 52
Unique qtr hrs #: 96


In [86]:
df_train["ORIGIN_CALL"].unique()

array([    0, 15138, 15935, ..., 20172,  2514, 28680], dtype=int64)

## Export

In [ ]:
df_train.head()
# df_test.head()

In [ ]:
df_test.head()

### Remove outliers

In [87]:
df_train = df_train[df_train["TARGET"] > 30]
df_train = df_train[df_train["TARGET"] < df_train["TARGET"].mean() + 5 * df_train["TARGET"].std()]

### [0, 1] Normalization

In [88]:
df_train["YEAR"] -= 2013
df_test["YEAR"] -= 2013

In [89]:
df_train.head()

,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,A,B,C,YEAR,WK_OF_YR,WK_DAY,QTR_HR,TARGET
0,0,0,367,0,0,1,0,25,6,68,345
1,0,7,371,0,1,0,0,25,6,68,285
2,0,0,204,0,0,1,0,25,6,68,975
3,0,0,330,0,0,1,0,25,6,68,645
4,0,0,217,0,0,1,0,25,6,68,435


#### Split training set

In [90]:
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_valid = df_train[int(0.9 * len(df_train)):]
df_train = df_train[:int(0.9 * len(df_train))]
df_train.to_csv("data/processed_trainv3.csv", index=False)
df_valid.to_csv("data/processed_validv3.csv", index=False)
df_test.to_csv("data/processed_testv3.csv", index=False)
